In [34]:
# Import libraries
from bs4 import BeautifulSoup
import csv
import json
import pandas as pd
import numpy as np
import re
import requests
import time
import warnings
import random
warnings.filterwarnings("ignore")

In [12]:
# Import csv file
hotels_path = "../data/properties_and_cities.csv"

In [13]:
# Create dataframe
hotels_df = pd.read_csv(hotels_path, usecols = ["item_id", "properties", "city"])

In [14]:
# Sanity check
hotels_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 927142 entries, 0 to 927141
Data columns (total 3 columns):
item_id       927142 non-null int64
properties    927142 non-null object
city          363557 non-null object
dtypes: int64(1), object(2)
memory usage: 21.2+ MB


In [15]:
hotels_df.head(100)

,item_id,properties,city
0,5101,Satellite TV|Golf Course|Airport Shuttle|Cosme...,"Mannheim, Germany"
1,5416,Satellite TV|Cosmetic Mirror|Safe (Hotel)|Tele...,"Passau, Germany"
2,5834,Satellite TV|Cosmetic Mirror|Safe (Hotel)|Tele...,"Bremen, Germany"
3,5910,Satellite TV|Sailing|Cosmetic Mirror|Telephone...,"Hamburg, Germany"
4,6066,Satellite TV|Sailing|Diving|Cosmetic Mirror|Sa...,"Stralsund, Germany"
5,6094,Satellite TV|Sailing|Safe (Hotel)|Telephone|Ho...,"Wolfsburg, Germany"
6,6288,Satellite TV|Airport Shuttle|Cosmetic Mirror|S...,"Bonn, Germany"
7,6358,Cosmetic Mirror|Safe (Hotel)|Telephone|Hotel|C...,"Dusseldorf, Germany"
8,6456,Satellite TV|Cosmetic Mirror|Safe (Hotel)|Tele...,"Cologne, Germany"
9,6561,Satellite TV|Golf Course|Cosmetic Mirror|Safe ...,"Dresden, Germany"


In [17]:
# # Create empty columns for the values that will be pulled through API
hotels_df["rating"] = np.nan
hotels_df["name"] = None

In [18]:
# Create item list
id_list = hotels_df["item_id"].tolist()

In [19]:
# List lenght
len(id_list)

927142

In [52]:
# Scraping trivago for additional information
for index, row in  hotels_df.head(100).iterrows():
    # Create URL and get html content
    idx = str(id_list[index])
    url = "https://www.trivago.com/kuta-92326/hotel/hotel-neo-kuta-legian-" + idx
    time.sleep(random.randrange(0,5))
    r = requests.get(url)
    if r.status_code == 200:
        c = r.content
        soup = BeautifulSoup(c, "html.parser")    
        hotel_name = soup.find("h1", class_="welcome-box__headline").text.strip()
        hotel_rating = float(soup.find("span", class_="rating_overall").strong.text.strip())
        hotel_city = soup.find("span", itemprop="addressLocality").text + ", " + soup.find("span", itemprop="addressCountry").text
        
        # Fill each column of the row with the values gotten from the API response
        hotels_df.at[index, "rating"] = hotel_rating
        hotels_df.at[index, "name"] = hotel_name
        hotels_df.at[index, "city"] = hotel_city
        print(str(index) + "," + str(idx) + "," + hotel_name + "," + hotel_city + "," + str(hotel_rating))
    else:
        print(r.status_code)           

0,5101,Best Western Plus Delta Park Hotel,Mannheim, Germany,7.8
1,5416,Hotel König,Passau, Germany,8.5
2,5834,Novum Hotel Bremer Haus Bremen,Bremen, Germany,8.1
3,5910,Aussen Alster,Hamburg, Germany,7.6
4,6066,Hotel Wyndham Stralsund HanseDom,Stralsund, Germany,7.8
5,6094,Centro Hotel Goya,Wolfsburg, Germany,7.6
6,6288,Akzent Hotel Am Hohenzollernplatz,Bonn, Germany,8.0
7,6358,Courtyard by Marriott Duesseldorf Hafen,Dusseldorf, Germany,8.1
8,6456,Hotel am Augustinerplatz,Cologne, Germany,7.9
9,6561,Wyndham Garden Dresden,Dresden, Germany,8.1
10,6659,Dorint Strandresort & Spa Sylt Westerland,Westerland, Germany,8.5
11,6853,Exagon Park,Can Picafort, Spain,8.2
12,7362,Hotel ValleMar,Puerto de la Cruz, Spain,8.4
13,7994,Holiday Inn San Francisco Airport,South San Francisco, USA,7.0
14,8476,The Buckingham Club,Chicago, USA,8.6
15,8810,Royalton,New York, USA,8.2
403
403
403
403
403
403
403
403
403
403
403
403
403
403
403
403
403
403
403
403
403
403
403
403
403
41,17450,Hotel Ripa Roma,Rome, 

AttributeError: 'NoneType' object has no attribute 'strong'